<a href="https://colab.research.google.com/github/lifeisbeautifu1/TensorFlow/blob/master/FireNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import the necessary packages
import os
# initialize the path to the fire and non-fire dataset directories
FIRE_PATH = os.path.sep.join(["drive", "MyDrive", "Fire"])
NON_FIRE_PATH = os.path.sep.join(["drive", "MyDrive", "Non-Fire"])
CLASSES = ["Non-Fire", "Fire"]

# define the size of the training and testing split
TRAIN_SPLIT = 0.75
TEST_SPLIT = 0.25
# define the initial learning rate, batch size, and number of epochs
INIT_LR = 1e-2
BATCH_SIZE = 64
NUM_EPOCHS = 50

# set the path to the serialized model after training
MODEL_PATH = os.path.sep.join(["output", "fire_detection.model"])
TRAINING_PLOT_PATH = os.path.sep.join(["output", "training_plot.png"])
OUTPUT_IMAGE_PATH = os.path.sep.join(["output", "examples"])
SAMPLE_SIZE = 50

In [ ]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
# import the necessary packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SeparableConv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense

class FireDetectionNet:
	@staticmethod
	def build(width, height, depth, classes):
		# initialize the model along with the input shape to be
		# "channels last" and the channels dimension itself
		model = Sequential()
		inputShape = (height, width, depth)
		chanDim = -1
    # CONV => RELU => POOL
		model.add(SeparableConv2D(16, (7, 7), padding="same",
			input_shape=inputShape))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
    # CONV => RELU => POOL
		model.add(SeparableConv2D(32, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		# (CONV => RELU) * 2 => POOL
		model.add(SeparableConv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(SeparableConv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
    # first set of FC => RELU layers
		model.add(Flatten())
		model.add(Dense(128))
		model.add(Activation("relu"))
		model.add(BatchNormalization())
		model.add(Dropout(0.5))
		# second set of FC => RELU layers
		model.add(Dense(128))
		model.add(Activation("relu"))
		model.add(BatchNormalization())
		model.add(Dropout(0.5))
		# softmax classifier
		model.add(Dense(classes))
		model.add(Activation("softmax"))
		# return the constructed network architecture
		return model

In [ ]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import cv2
import sys

In [ ]:
def load_dataset(datasetPath):
	# grab the paths to all images in our dataset directory, then
	# initialize our lists of images
	imagePaths = list(paths.list_images(datasetPath))
	data = []
	# loop over the image paths
	for imagePath in imagePaths:
		# load the image and resize it to be a fixed 128x128 pixels,
		# ignoring aspect ratio
		image = cv2.imread(imagePath)
		if image is None:
			continue
		image = cv2.resize(image, (128, 128))
		# add the image to the data lists
		data.append(image)
	# return the data list as a NumPy array
	return np.array(data, dtype="float32")

In [ ]:
print("[INFO] loading data...")
fireData = load_dataset(FIRE_PATH)
nonFireData = load_dataset(NON_FIRE_PATH)
# construct the class labels for the data
fireLabels = np.ones((fireData.shape[0],))
nonFireLabels = np.zeros((nonFireData.shape[0],))
# stack the fire data with the non-fire data, then scale the data
# to the range [0, 1]
data = np.vstack([fireData, nonFireData])
labels = np.hstack([fireLabels, nonFireLabels])
data /= 255

[INFO] loading data...


KeyboardInterrupt: ignored

In [ ]:
# perform one-hot encoding on the labels and account for skew in the
# labeled data
labels = to_categorical(labels, num_classes=2)
classTotals = labels.sum(axis=0)
classWeight = classTotals.max() / classTotals
print(classWeight)
# construct the training and testing split
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=TEST_SPLIT, random_state=42)

[1.        1.3465458]


In [ ]:
aug = ImageDataGenerator(
	rotation_range=30,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")
# initialize the optimizer and model
print("[INFO] compiling model...")
opt = SGD(learning_rate=INIT_LR, momentum=0.9,
	decay=INIT_LR / NUM_EPOCHS)
model = FireDetectionNet.build(width=128, height=128, depth=3,
	classes=2)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])

[INFO] compiling model...


In [ ]:
print("[INFO] training network...")
H = model.fit_generator(
	aug.flow(trainX, trainY, batch_size=BATCH_SIZE),
	validation_data=(testX, testY),
	steps_per_epoch=trainX.shape[0] // BATCH_SIZE,
	epochs=NUM_EPOCHS,
  verbose=1)

[INFO] training network...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  import sys


Epoch 1/50
48/48 [==============================] - 22s 255ms/step - loss: 0.7053 - accuracy: 0.6994 - val_loss: 0.8177 - val_accuracy: 0.4537
Epoch 2/50
48/48 [==============================] - 12s 245ms/step - loss: 0.5235 - accuracy: 0.7802 - val_loss: 1.8518 - val_accuracy: 0.4537
Epoch 3/50
48/48 [==============================] - 12s 250ms/step - loss: 0.4747 - accuracy: 0.7957 - val_loss: 1.3667 - val_accuracy: 0.4537
Epoch 4/50
48/48 [==============================] - 12s 241ms/step - loss: 0.4331 - accuracy: 0.8062 - val_loss: 1.2903 - val_accuracy: 0.4537
Epoch 5/50
48/48 [==============================] - 12s 241ms/step - loss: 0.4036 - accuracy: 0.8282 - val_loss: 1.2461 - val_accuracy: 0.4537
Epoch 6/50
48/48 [==============================] - 12s 243ms/step - loss: 0.3962 - accuracy: 0.8325 - val_loss: 1.1538 - val_accuracy: 0.4566
Epoch 7/50
48/48 [==============================] - 12s 241ms/step - loss: 0.3743 - accuracy: 0.8456 - val_loss: 0.6862 - val_accuracy: 0.5608

In [ ]:
# evaluate the network and show a classification report
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=BATCH_SIZE)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=CLASSES))
# serialize the model to disk
print("[INFO] serializing network to '{}'...".format(MODEL_PATH))
model.save(MODEL_PATH)
# construct a plot that plots and saves the training history
N = np.arange(0, NUM_EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["accuracy"], label="train_acc")
plt.plot(N, H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(TRAINING_PLOT_PATH)

[INFO] evaluating network...
17/17 [==============================] - 0s 15ms/step
              precision    recall  f1-score   support

    Non-Fire       0.95      0.96      0.95       566
        Fire       0.95      0.94      0.94       470

    accuracy                           0.95      1036
   macro avg       0.95      0.95      0.95      1036
weighted avg       0.95      0.95      0.95      1036

[INFO] serializing network to 'output/fire_detection.model'...


In [ ]:
# import the necessary packages
from tensorflow.keras.models import load_model
from imutils import paths
import numpy as np
import imutils
import random
import cv2
import os
# load the trained model from disk
print("[INFO] loading model...")
model = load_model(MODEL_PATH)
# model = load_model(os.path.sep.join(["drive", "MyDrive", "output", "fire_detection.model"]))
# grab the paths to the fire and non-fire images, respectively
print("[INFO] predicting...")
firePaths = list(paths.list_images(FIRE_PATH))
nonFirePaths = list(paths.list_images(NON_FIRE_PATH))
# combine the two image path lists, randomly shuffle them, and sample
# them
imagePaths = firePaths + nonFirePaths
random.shuffle(imagePaths)
imagePaths = imagePaths[:SAMPLE_SIZE]

[INFO] loading model...
[INFO] predicting...


In [ ]:
for (i, imagePath) in enumerate(imagePaths):
	# load the image and clone it
	print(imagePath)
	image = cv2.imread(imagePath)
	output = image.copy()
	# resize the input image to be a fixed 128x128 pixels, ignoring
	# aspect ratio
	image = cv2.resize(image, (128, 128))
	image = image.astype("float32") / 255.0
		
	# make predictions on the image
	preds = model.predict(np.expand_dims(image, axis=0))[0]
	j = np.argmax(preds)
	print(j)
	label = CLASSES[j]
	print(label)
	# draw the activity on the output frame
	text = label if label == "Non-Fire" else "WARNING! Fire!"
	output = imutils.resize(output, width=500)
	cv2.putText(output, text, (35, 50), cv2.FONT_HERSHEY_SIMPLEX,
		1.25, (0, 255, 0), 5)
	# write the output image to disk	 
	filename = "{}.png".format(i)
	p = os.path.sep.join([OUTPUT_IMAGE_PATH, filename])
	print(p)
	cv2.imwrite(p, output)

drive/MyDrive/Non-Fire/building58_right_day.jpg


KeyboardInterrupt: ignored

In [ ]:
from google.colab import files
uploaded = files.upload()

imagePath = list(uploaded.keys())[0]


image = cv2.imread(imagePath)
output = image.copy()

image = cv2.resize(image, (128, 128))
image = image.astype("float32") / 255.0

# make predictions on the image
preds = model.predict(np.expand_dims(image, axis=0))[0]
j = np.argmax(preds)
label = CLASSES[j]
print(label)
# draw the activity on the output frame
text = label if label == "Non-Fire" else "WARNING! Fire!"
output = imutils.resize(output, width=500)
cv2.putText(output, text, (35, 50), cv2.FONT_HERSHEY_SIMPLEX,
		1.25, (0, 255, 0), 5)
# write the output image to disk	 
filename = "foresttest3.png"
p = os.path.sep.join([filename])
cv2.imwrite(p, output)






Saving concept-forest-fires-burning-forest-top-view-aerial-view-white-smoke-is-coming-from-forest_431724-5578.jpg to concept-forest-fires-burning-forest-top-view-aerial-view-white-smoke-is-coming-from-forest_431724-5578.jpg
1/1 [==============================] - 0s 22ms/step
Non-Fire


True

In [ ]:
from tensorflow.keras.models import load_model
from collections import deque
import numpy as np
import cv2


# load the trained model from disk
print("[INFO] loading model...")
model = load_model(MODEL_PATH)


# initialize the image mean for mean subtraction along with the
# predictions queue
# mean = np.array([123.68, 116.779, 103.939][::1], dtype="float32")
Q = deque(maxlen=128)

[INFO] loading model...


In [ ]:
from google.colab import files
uploaded = files.upload()

videoPath = list(uploaded.keys())[0]

# initialize the video stream, pointer to output video file, and
# frame dimensions
vs = cv2.VideoCapture(videoPath)
# vs = cv2.VideoCapture(videoPath)
writer = None
(W, H) = (None, None)
# loop over frames from the video file stream
while True:
	# read the next frame from the file
	(grabbed, frame) = vs.read()
	# if the frame was not grabbed, then we have reached the end
	# of the stream
	if not grabbed:
		break
	# if the frame dimensions are empty, grab them
	if W is None or H is None:
		(H, W) = frame.shape[:2]
  # clone the output frame, then convert it from BGR to RGB
	# ordering, resize the frame to a fixed 224x224, and then
	# perform mean subtraction
	output = frame.copy()
	# frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
	frame = cv2.resize(frame, (128, 128)).astype("float32") / 255.0
	# frame -= mean
  # clone the output frame, then convert it from BGR to RGB
	# ordering, resize the frame to a fixed 224x224, and then
	# perform mean subtraction
	preds = model.predict(np.expand_dims(frame, axis=0))[0]
	# frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
	Q.append(preds)
	results = np.array(Q).mean(axis=0)
  # clone the output frame, then convert it from BGR to RGB
	# ordering, resize the frame to a fixed 224x224, and then
	# perform mean subtraction
	i = np.argmax(results)
	# frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
	label = CLASSES[i]
	print(label)
 # draw the activity on the output frame
	text = "activity: {}".format(label)
	cv2.putText(output, text, (35, 50), cv2.FONT_HERSHEY_SIMPLEX,
		1.25, (0, 255, 0), 5)
	# check if the video writer is None
	if writer is None:
		# initialize our video writer
		fourcc = cv2.VideoWriter_fourcc(*"MJPG")
		writer = cv2.VideoWriter(os.path.sep.join(['output2.avi']), fourcc, 30,
			(W, H), True)
	# write the output frame to disk
	writer.write(output)
	
print("[INFO] cleaning up...")
writer.release()
vs.release()
	

Saving pexels-charles-parker-5827642.mp4 to pexels-charles-parker-5827642.mp4
1/1 [==============================] - 0s 20ms/step
Fire
1/1 [==============================] - 0s 18ms/step
Fire
1/1 [==============================] - 0s 22ms/step
Fire
1/1 [==============================] - 0s 18ms/step
Fire
1/1 [==============================] - 0s 30ms/step
Fire
1/1 [==============================] - 0s 22ms/step
Fire
1/1 [==============================] - 0s 23ms/step
Fire
1/1 [==============================] - 0s 24ms/step
Fire
1/1 [==============================] - 0s 24ms/step
Fire
1/1 [==============================] - 0s 44ms/step
Fire
1/1 [==============================] - 0s 22ms/step
Fire
1/1 [==============================] - 0s 28ms/step
Fire
1/1 [==============================] - 0s 25ms/step
Fire
1/1 [==============================] - 0s 21ms/step
Fire
1/1 [==============================] - 0s 20ms/step
Fire
1/1 [==============================] - 0s 31ms/step
Fire
1/1 [=====